In [2]:
import numpy as np
import pandas as pd
import random
from kaggle_environments import evaluate
from kaggle_environments.envs.rps.utils import get_score # импортируем функцию получения счёта

 В нашей игре присвоим обозначения для ходов игроков:
<ul>
    <li><b>Камень</b> - 0</li>
    <li><b>Бумага</b> - 1</li>
    <li><b>Ножницы</b> - 2</li>
</ul>

Опишем самые банальные стратегии для агентов:

In [201]:
def rock(observation, configuration):
    return 0 # всегда ход - камень

def paper(observation, configuration):
    return 1 # всегда ход - бумага

def scissors(observation, configuration):
    return 2 # всегда ход - ножницы

Также реализуем, трёх игроков, которые выбирают между двумя ходами

In [202]:
def rock_paper(observation, configuration):
    return random.choice([0,1]) # случайный ход - камень или бумага

def paper_scissors(observation, configuration):
    return random.choice([1,2])# случайный ход - бумага или ножницы

def scissors_rock(observation, configuration):
    return random.choice([0,2]) # случайный ход - ножницы или камень

Опишем игрока, который просто случайно делает выбор.

In [203]:
def rand(observation, configuration):
    return random.choice([0,1,2])

Также будет игрок, который в зависимости от попадания нормальной случайной величины в интервал от 0 до 3 будет делать ход.
$$\xi = N(m,\sigma^2),$$
где $m = 1,5$ - математическое ожидание;
$\sigma = \sqrt{\sigma^2} = 0,5$ - стандартное отклонение.

In [204]:
def norm(observation, configuration):
    x = random.normalvariate(1.5, 0.5)
    if x < 1: # если случайное число попадает от 0 до 1, то ход - камень
        return 0
    elif x < 2: # если случайное число попадает от 1 до 2, то ход - бумага
        return 1
    else: return 2 # если случайное число попадает от 2 до 3, то ход - ножницы

Игрок, который в зависимости от попадания равномерной случайной величины в интервал от 0 до 3 будет делать ход.
$$\xi = R(a, b),$$
где $a = 0$, $b = 3$ - границы.

In [205]:
def uniform(observation, configuration):
    x = random.uniform(0, 3)
    if x < 1: # если случайное число попадает от 0 до 1, то ход - камень
        return 0
    elif x < 2: # если случайное число попадает от 1 до 2, то ход - бумага
        return 1
    else: return 2 # если случайное число попадает от 2 до 3, то ход - ножницы

Игрок, который будет поторять ход оппонента с предыдущей игры.

In [206]:
def copy(observation, configuration):
    if observation.step == 0:
        return random.choice([0, 1, 2])
    else: return observation.lastOpponentAction

Если есть, игрок копирующий последнийх ход оппонента, то пусть будет игрок, который будет ставить выигрышный ход к последнему ходу оппонента.

In [207]:
def opposite(observation, configuration):
    if observation.step == 0:
        return random.choice([0, 1, 2])
    elif observation.lastOpponentAction == 0: # если камень, то выбирает бумагу
        return 1
    elif observation.lastOpponentAction == 1: # если бумага, то выбирает ножницы
        return 2
    else: return 0

Игрок, который будет выбирать наиболее частый ход соперника за предыдущие игры.

In [217]:
choice_opp = {} # словарь с ходами противника

def freq(observation, configuration):
    global choice_opp # делаем глобальную переменную, так как функция будет вызываться много раз
    if observation.step == 0:
        choice_opp.clear()
        return random.choice([0,1,2])
    do = observation.lastOpponentAction
    choice_opp[do] = choice_opp.get(do, 0) + 1
    """
    Ищем количество ходов жеста с предыдущей игры, если не было то мы заводим запись в словаре, если нашли то засчитываем как очередной ход.
    """
    return max(choice_opp, key=lambda x: choice_opp.get(x)) #возвращает частый ход

Игрок, который будет противодействовать наиболее частому ходу соперника за предыдущие игры.

In [221]:
choice_opp2 = {} # словарь с ходами противника

def freq_opp(observation, configuration):
    global choice_opp2 # делаем глобальную переменную, так как функция будет вызываться много раз
    if observation.step == 0:
        choice_opp2.clear()
        return random.choice([0,1,2])
    do = observation.lastOpponentAction
    choice_opp2[do] = choice_opp2.get(do, 0) + 1
    m = max(choice_opp, key=lambda x: choice_opp.get(x)) # наиболее частый ход

    """
    Ставит противоположный жест наиболее частому ходу противника.
    """

    if m == 0:
        return 1
    elif m == 1:
        return 2
    else: return 0


Игрок, который будет выбирать наиболее удачный ход противника.

In [222]:
choice_opp1 = {} # словарь с ходами противника
last_choice = None #свой последний ход

def luck(observation, configuration):
    global choice_opp1, last_choice # делаем глобальную переменную, так как функция будет вызываться много раз
    if observation.step == 0:
        choice_opp1.clear()
        last_choice = random.choice([0,1,2])
    else:
        do = observation.lastOpponentAction
        choice_opp1[do] = choice_opp1.get(do, 0) + get_score(last_choice, observation.lastOpponentAction)[1]
    """
    Ищем количество ходов жеста с предыдущей игры, если не было то мы заводим запись в словаре, если нашли то считаем счёт.
    """
    print(choice_opp1)
    return max(choice_opp1, key=lambda x: choice_opp1.get(x)) #возвращает частый ход

In [224]:
agents = {
    "rock_player": rock,
    "paper_player": paper,
    "scissors_player": scissors,
    "rock_paper_player": rock_paper,
    "paper_scissors_player": paper_scissors,
    "scissors_rock_player": scissors_rock,
    "random_player": rand,
    "normal_player": norm,
    "uniform_player": uniform,
    "copy_player": copy,
    "opposite_player": opposite,
    "frequency_player": freq,
    "frequency_opposite_player": freq_opp,
    "lucky_player": luck,
}
players = ["rock_player", "paper_player", "scissors_player", "rock_paper_player", "paper_scissors_player", "scissors_rock_player",  "random_player", "normal_player", "uniform_player", "copy_player", "opposite_player", "frequency_player","frequency_opposite_player", "lucky_player"]

In [228]:
n = len(players)
game_statistics = np.zeros((n, n)) # таблица счёта
for i in players:
    for j in players:
        if i != j:
            result = evaluate(
                "rps", # игра
                agents=[agents[i], agents[j]], # игроки
                configuration={"episodeSteps": 100, # количество игр
                               "tieRewardThreshold": 1} # число очков, с которого начинается победа
            )
            game_statistics[players.index(i)][players.index(j)] = result[0][0]
            game_statistics[players.index(j)][players.index(i)] = result[0][1]

game_statistics = pd.DataFrame(game_statistics, index=players, columns=players)
"""""
Здесь смотреть противостояния нужно по строкам.
"""""
game_statistics.head(14)

,rock_player,paper_player,scissors_player,rock_paper_player,paper_scissors_player,scissors_rock_player,random_player,normal_player,uniform_player,copy_player,opposite_player,frequency_player,frequency_opposite_player,lucky_player
rock_player,0.0,-99.0,99.0,-54.0,-1.0,51.0,-4.0,-62.0,-3.0,-1.0,-99.0,1.0,1.0,1.0
paper_player,99.0,0.0,-99.0,49.0,-56.0,15.0,-9.0,-9.0,3.0,-1.0,-99.0,1.0,1.0,1.0
scissors_player,-99.0,99.0,0.0,-13.0,53.0,-57.0,2.0,59.0,14.0,0.0,-99.0,1.0,1.0,1.0
rock_paper_player,54.0,-49.0,13.0,0.0,-30.0,8.0,7.0,-20.0,9.0,-1.0,-35.0,1.0,1.0,1.0
paper_scissors_player,1.0,56.0,-53.0,30.0,0.0,-8.0,-3.0,39.0,4.0,-1.0,-34.0,1.0,1.0,1.0
scissors_rock_player,-51.0,-15.0,57.0,-8.0,8.0,0.0,-3.0,11.0,0.0,0.0,-25.0,1.0,1.0,1.0
random_player,4.0,9.0,-2.0,-7.0,3.0,3.0,0.0,8.0,2.0,10.0,1.0,1.0,1.0,1.0
normal_player,62.0,9.0,-59.0,20.0,-39.0,-11.0,-8.0,0.0,14.0,-7.0,-18.0,1.0,1.0,1.0
uniform_player,3.0,-3.0,-14.0,-9.0,-4.0,0.0,-2.0,-14.0,0.0,10.0,-5.0,1.0,1.0,1.0
copy_player,1.0,1.0,0.0,1.0,1.0,0.0,-10.0,7.0,-10.0,0.0,-50.0,1.0,1.0,1.0


Определим победителя.

In [290]:
table_winners_f = np.zeros((n, n))
table_winners_f = pd.DataFrame(table_winners_f, index=players, columns=players)
for i in range(len(game_statistics)):
    for j in range(len(game_statistics.columns)):
        table_winners_f.iloc[i,j] = (1 if game_statistics.iloc[i,j] > 0 else 0) # здесь мы заменяем количество выигранных партий на победу или поражение
table_winners = table_winners_f.sum(axis=0) # считаем количество побед каждого игрока и сохраняем статистику побед
print("Победителем становится {}".format(table_winners.sort_values(axis=0, ascending=0).index[0])) # выводим победителя

Победителем становится lucky_player
